In [15]:
import tkinter as tk
from tkinter.ttk import Frame, Label, Entry, Button, Style, Combobox, Treeview, Scrollbar
from tkinter import BOTH, END, messagebox, Toplevel
from datetime import datetime
import csv
import os

print("Imports loaded")

Imports loaded


In [16]:
class LoanCalculatorApp(Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        self.history_file = "loan_history.csv"  # file where we save results
        self.init_ui()

    def init_ui(self):
        """Build all the labels, text boxes and buttons."""
        self.pack(fill=BOTH, expand=True)

        style = Style()
        style.theme_use("default")
        style.configure("Main.TButton", foreground="white", background="blue")
        style.configure("Exit.TButton", foreground="red")
        style.configure("Secondary.TButton", foreground="black")

        xpos = 40
        ypos = 20
        xpos2 = xpos + 160

        # Amount
        Label(self, text="Amount", foreground="#ff0000",
              background="light blue", font="Arial 9").place(x=xpos, y=ypos)
        self.txt_amount = Entry(self)
        self.txt_amount.place(x=xpos2, y=ypos, width=130)

        # Interest rate
        ypos += 30
        Label(self, text="Rate (%)", foreground="#ff0000",
              background="light blue", font="Arial 9").place(x=xpos, y=ypos)
        self.txt_rate = Entry(self)
        self.txt_rate.place(x=xpos2, y=ypos, width=130)

        # Duration value
        ypos += 30
        Label(self, text="Duration", foreground="#ff0000",
              background="light blue", font="Arial 9").place(x=xpos, y=ypos)
        self.txt_duration = Entry(self)
        self.txt_duration.place(x=xpos2, y=ypos, width=60)

        # Duration unit (Months / Years) 
        self.cbo_unit = Combobox(self, values=["Months", "Years"], state="readonly")
        self.cbo_unit.current(0)  # default = Months
        self.cbo_unit.place(x=xpos2 + 70, y=ypos, width=60)

        # Monthly payment (read-only)
        ypos += 30
        Label(self, text="Monthly Payment", foreground="#ff0000",
              background="yellow", font="Arial 9").place(x=xpos, y=ypos)
        self.txt_monthly = Entry(self, state="readonly")
        self.txt_monthly.place(x=xpos2, y=ypos, width=130)

        # Total payment (read-only) 
        ypos += 30
        Label(self, text="Total Payment", foreground="#ff0000",
              background="yellow", font="Arial 9").place(x=xpos, y=ypos)
        self.txt_total = Entry(self, state="readonly")
        self.txt_total.place(x=xpos2, y=ypos, width=130)

        # Total interest (read-only) NEW FEATURE 
        ypos += 30
        Label(self, text="Total Interest", foreground="#ff0000",
              background="yellow", font="Arial 9").place(x=xpos, y=ypos)
        self.txt_interest = Entry(self, state="readonly")
        self.txt_interest.place(x=xpos2, y=ypos, width=130)

        # Buttons row 1 
        ypos += 40
        btn_calc = Button(self, text="Calculate", style="Main.TButton",
                          command=self.on_calculate)
        btn_calc.place(x=xpos, y=ypos)

        btn_clear = Button(self, text="Clear", style="Secondary.TButton",
                           command=self.on_clear)
        btn_clear.place(x=xpos + 120, y=ypos)

        # Buttons row 2 
        ypos += 35
        btn_save = Button(self, text="Save Result", style="Secondary.TButton",
                          command=self.on_save)
        btn_save.place(x=xpos, y=ypos)

        btn_view = Button(self, text="View History", style="Secondary.TButton",
                          command=self.on_view_history)   # NEW FEATURE
        btn_view.place(x=xpos + 120, y=ypos)

        # Buttons row 3 
        ypos += 35
        btn_clear_hist = Button(self, text="Clear History", style="Secondary.TButton",
                                command=self.on_clear_history)  # NEW FEATURE
        btn_clear_hist.place(x=xpos, y=ypos)

        btn_exit = Button(self, text="Exit", style="Exit.TButton",
                          command=self.on_exit)
        btn_exit.place(x=xpos + 120, y=ypos)

    # helper methods 

    def _get_float(self, entry: Entry, field_name: str):
        """Validate that entry is not empty and numeric; return float or raise ValueError."""
        text = entry.get().strip()
        if not text:
            raise ValueError(f"{field_name} is required.")
        try:
            value = float(text)
        except ValueError:
            raise ValueError(f"{field_name} must be a number.")
        if value <= 0:
            raise ValueError(f"{field_name} must be greater than zero.")
        return value

    def _set_readonly(self, entry: Entry, value: str):
        """Set text in a readonly Entry widget."""
        entry.configure(state="normal")
        entry.delete(0, END)
        entry.insert(0, value)
        entry.configure(state="readonly")

    # button actions

    def on_calculate(self):
        """
        Calculate monthly and total loan payment.

        Formula:
            monthly_rate = rate / 1200
            monthly_payment = [A * r * (1+r)^n] / [(1+r)^n - 1]
        """
        try:
            amount = self._get_float(self.txt_amount, "Amount")
            rate = self._get_float(self.txt_rate, "Rate")
            duration_val = self._get_float(self.txt_duration, "Duration")
        except ValueError as e:
            messagebox.showerror("Input error", str(e))
            return

        # Convert duration to months depending on selected unit
        unit = self.cbo_unit.get()
        if unit == "Years":
            duration = duration_val * 12
        else:
            duration = duration_val

        monthly_rate = rate / 1200.0  # convert % per year to decimal per month

        if duration <= 0 or monthly_rate <= 0:
            messagebox.showerror("Input error",
                                 "Rate and Duration must be greater than zero.")
            return

        # Standard loan formula
        monthly_payment = (
            amount * monthly_rate * (1 + monthly_rate) ** duration
            / ((1 + monthly_rate) ** duration - 1)
        )
        total_payment = monthly_payment * duration
        total_interest = total_payment - amount  # NEW FEATURE

        self._set_readonly(self.txt_monthly, f"{monthly_payment:0.2f}")
        self._set_readonly(self.txt_total, f"{total_payment:0.2f}")
        self._set_readonly(self.txt_interest, f"{total_interest:0.2f}")

    def on_clear(self):
        """Clear all inputs and outputs."""
        for entry in [self.txt_amount, self.txt_rate, self.txt_duration]:
            entry.delete(0, END)
        self.cbo_unit.current(0)  # reset to Months
        for entry in [self.txt_monthly, self.txt_total, self.txt_interest]:
            self._set_readonly(entry, "")

    def on_save(self):
        """Save last calculation to loan_history.csv in the same folder as this notebook."""
        monthly = self.txt_monthly.get().strip()
        total = self.txt_total.get().strip()
        interest = self.txt_interest.get().strip()
        if not monthly or not total:
            messagebox.showwarning("Nothing to save",
                                   "Calculate a loan first before saving.")
            return

        amount = self.txt_amount.get().strip()
        rate = self.txt_rate.get().strip()
        duration = self.txt_duration.get().strip()
        unit = self.cbo_unit.get()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        is_new = not os.path.exists(self.history_file)
        with open(self.history_file, mode="a", newline="") as f:
            writer = csv.writer(f)
            if is_new:
                writer.writerow(
                    [
                        "timestamp", "amount", "rate",
                        "duration_value", "duration_unit",
                        "monthly_payment", "total_payment", "total_interest"
                    ]
                )
            writer.writerow(
                [timestamp, amount, rate, duration, unit, monthly, total, interest]
            )

        messagebox.showinfo("Saved",
                            f"Calculation saved to {self.history_file}")

    def on_view_history(self):
        """Open a popup window showing all saved history in a table."""
        if not os.path.exists(self.history_file):
            messagebox.showinfo("No history",
                                "No history file found yet. Save a result first.")
            return

        win = Toplevel(self)
        win.title("Loan History")
        win.geometry("800x300")

        cols = (
            "timestamp", "amount", "rate",
            "duration", "unit", "monthly", "total", "interest"
        )
        tree = Treeview(win, columns=cols, show="headings")

        headings = [
            ("timestamp", "Timestamp"),
            ("amount", "Amount"),
            ("rate", "Rate (%)"),
            ("duration", "Duration"),
            ("unit", "Unit"),
            ("monthly", "Monthly Payment"),
            ("total", "Total Payment"),
            ("interest", "Total Interest"),
        ]

        for col, text in headings:
            tree.heading(col, text=text)
            tree.column(col, width=90, anchor="center")

        # Scrollbar
        vsb = Scrollbar(win, orient="vertical", command=tree.yview)
        tree.configure(yscrollcommand=vsb.set)

        tree.pack(side="left", fill="both", expand=True)
        vsb.pack(side="right", fill="y")

        # Load data from CSV
        with open(self.history_file, newline="") as f:
            reader = csv.reader(f)
            header = next(reader, None)  # skip header
            for row in reader:
                # Expect 8 columns as written in on_save
                if len(row) == 8:
                    tree.insert("", END, values=row)

    def on_clear_history(self):
        """Delete the history CSV file after user confirmation."""
        if not os.path.exists(self.history_file):
            messagebox.showinfo("No history",
                                "There is no history file to delete.")
            return

        if messagebox.askokcancel(
            "Clear history",
            "Are you sure you want to permanently delete all saved history?"
        ):
            try:
                os.remove(self.history_file)
                messagebox.showinfo("Cleared",
                                    "Loan history has been deleted.")
            except Exception as e:
                messagebox.showerror("Error", f"Could not delete file: {e}")

    def on_exit(self):
        """Close the application window."""
        if messagebox.askokcancel("Exit", "Close application?"):
            self.parent.destroy()

print("Enhanced LoanCalculatorApp class defined")

Enhanced LoanCalculatorApp class defined


In [19]:
# App launch
root = tk.Tk()
root.title("Loan Calculator Plus (Enhanced)")
root.geometry("380x360")

app = LoanCalculatorApp(root)

print("Starting Tkinter mainloop... a window should appear now.")
root.mainloop()

Starting Tkinter mainloop... a window should appear now.
